In [1]:
import pandas as pd
import numpy as np

In [2]:
def clean_up(phrase):
    phrase = phrase.lstrip()
    phrase = phrase.rstrip()
    phrase = phrase.upper()
    return phrase

In [3]:
supplier = pd.read_excel('files/bids_input.xlsx',  sheet_name='lists')
supplier.dtypes

isbn           int64
cost         float64
sup_qty      float64
Warehouse     object
dtype: object

In [4]:
#renaming headers
supplier.columns = map(str.lower, supplier.columns)

#Cleaning Values in columns
supplier = supplier.fillna('N/A')
supplier['isbn'] = supplier['isbn'].astype(str)
for col in ['isbn','warehouse']:
    supplier[col] = supplier.apply(lambda x: clean_up(x[col]), axis =1)
supplier['cost'] = supplier['cost'].round(decimals=2)
#adding level value to warehuse
warehouse_level = {'TB': 1,'AW':2,'Transit':3,'FBA':4}
supplier['level'] = supplier['warehouse'].map(warehouse_level)

#Sorting the lists
supplier = supplier.sort_values(['level','cost'], ascending=[True,True]).reset_index(drop=True)
supplier.head()

,isbn,cost,sup_qty,warehouse,level
0,9781337118606,7.22,20.0,AW,2
1,9781305655522,9.54,25.0,AW,2
2,9781337407724,9.57,25.0,AW,2
3,9781337407731,9.57,20.0,AW,2
4,9781337407779,9.57,2.0,AW,2


In [5]:
customer = pd.read_excel('files/bids_input.xlsx',  sheet_name='customers')
customer.dtypes

isbn          int64
customer     object
price       float64
cust_qty      int64
dtype: object

In [6]:
#Fix headers name
customer.columns = map(str.lower, customer.columns)
customer.rename(columns={'isbn13': 'isbn','price':'sale_price'}, inplace = True)

#cleaning values in columns
customer = customer.fillna('N/A')
customer['isbn'] = customer['isbn'].astype(str)
for col in ['isbn', 'customer']:
    customer[col] = customer.apply(lambda x: clean_up(x[col]), axis =1)
customer['sale_price'] = customer['sale_price'].round(decimals=2)
customer.rename(columns={'isbn13': 'isbn'}, inplace = True)
customer['cust_qty'] =customer['cust_qty'].round(decimals = 0)
customer.head()

,isbn,customer,sale_price,cust_qty
0,9780323374620,BOOKBYTE,156.01,5
1,9781564240071,NBC,151.47,13
2,9781585286157,BOOKBYTE,146.06,12
3,9781617210006,BOOKBYTE,141.55,4
4,9781642428087,BOOKBYTE,141.25,8


In [7]:
customer = customer.sort_values(['sale_price'], ascending=False).reset_index(drop=True)
customer.head()

,isbn,customer,sale_price,cust_qty
0,9780323374620,BOOKBYTE,156.01,5
1,9781564240071,NBC,151.47,13
2,9781585286157,BOOKBYTE,146.06,12
3,9781617210006,BOOKBYTE,141.55,4
4,9781642428087,BOOKBYTE,141.25,8


In [8]:
bid = pd.DataFrame()
for cust_ind in list(customer[customer['isbn'].isin(list(supplier['isbn']))].index.values):
    for su_ind in list(supplier[supplier['isbn'].isin(list(customer['isbn']))].index.values):
        if (supplier.iloc[su_ind, :]['isbn'] == customer.iloc[cust_ind, :]['isbn']) and (supplier.iloc[su_ind, :]['sup_qty'] > 0) and (customer.iloc[cust_ind, :]['cust_qty'] > 0):
            row_bid = pd.merge(supplier.iloc[[su_ind], :], customer.iloc[[cust_ind], :], how='left', on = 'isbn' )
            row_bid['qty'] = row_bid[['sup_qty','cust_qty']].min( axis = 1, skipna = True)
            row_bid['unit_diff'] = (customer.iloc[cust_ind, :]['sale_price'] - supplier.iloc[su_ind, :]['cost'])
            supplier.loc[su_ind,'sup_qty'] = supplier.loc[su_ind,'sup_qty'] - row_bid.iloc[0,:]['qty']
            customer.loc[cust_ind,'cust_qty'] = customer.loc[cust_ind,'cust_qty'] - row_bid.iloc[0,:]['qty']
            bid = bid.append(row_bid)

In [19]:
bid['decision'] = np.where(bid['unit_diff'] >= 0, 'YES', 'COUNTER')
bid['bid_price'] = np.where(bid['unit_diff'] >= 0, bid['sale_price'],bid['cost'])
bid = bid.sort_values(['decision', 'customer'], ascending = [False, True])
bid.head()

,isbn,cost,sup_qty,warehouse,level,customer,sale_price,cust_qty,qty,unit_diff,decision,bid_price
0,9780205535859,93.67,1.0,FBA,4,BOOKBYTE,105.89,10.0,1.0,12.22,YES,105.89
0,9780134602806,88.31,8.0,FBA,4,BOOKBYTE,103.86,14.0,8.0,15.55,YES,103.86
0,9781319114671,96.74,16.0,AW,2,BOOKBYTE,100.48,5.0,5.0,3.74,YES,100.48
0,9780134787961,78.25,16.0,FBA,4,BOOKBYTE,90.17,6.0,6.0,11.92,YES,90.17
0,9780071717595,80.19,21.0,FBA,4,BOOKBYTE,87.77,8.0,8.0,7.58,YES,87.77


In [20]:

with pd.ExcelWriter('files/bids_results.xlsx') as writer:
    bid.to_excel(writer,sheet_name = 'bid', index= False)
    supplier.to_excel(writer, sheet_name = 'supplier', index= False)
    customer.to_excel(writer, sheet_name = 'customer', index= False)


In [21]:
## Upload SO and Item Master
customer_results = bid.groupby(['isbn','customer', 'decision'], as_index= False).agg({'bid_price': 'mean', 'qty':'sum'})
customer_results.head()

,isbn,customer,decision,bid_price,qty
0,9780060000387,BOOKBYTE,COUNTER,29.66,22.0
1,9780060000387,NBC,COUNTER,29.66,31.0
2,9780071390118,NBC,COUNTER,54.20,1.0
3,9780071624947,BOOKBYTE,COUNTER,20.39,28.0
4,9780071624978,BOOKBYTE,COUNTER,60.49,1.0


In [22]:
with pd.ExcelWriter('files/bids_customer_results.xlsx') as writer:
    for name in list(customer_results['customer'].unique()):
        customer_results[customer_results['customer'] == name].to_excel(writer, sheet_name= name, index =False)

In [23]:
so = bid.groupby(['isbn','customer', 'decision','warehouse'], as_index= False).agg({'bid_price': 'mean', 'qty':'sum'})
inv_warehouse = bid.groupby(['isbn', 'decision','warehouse'], as_index= False).agg({'qty':'sum'})
recalls = so[so['warehouse'] == 'FBA']
recalls['customer'] = np.where((recalls['customer'] == 'BOOKBYTE') | (recalls['customer'] == 'BBA'), recalls['customer'], 'BXC')
recalls = recalls.groupby(['isbn', 'decision', 'warehouse'], as_index= False).agg({'qty':'sum'})

In [25]:
with pd.ExcelWriter('files/bids_operations.xlsx') as writer:
    so.to_excel(writer, sheet_name= 'so', index =False)
    inv_warehouse.to_excel(writer, sheet_name= 'inv_per_warehouse', index =False)
    recalls.to_excel(writer, sheet_name= 'recalls', index =False)